In [1]:
import pandas as pd
df = pd.read_csv('/Users/tusharbhatt/Desktop/Python for Finance/BTC-USDT.csv')
df

,date,open,high,low,close,volume
0,1502942400000,4261.48,4313.62,4261.32,4308.83,47.181009
1,1502946000000,4308.83,4328.69,4291.37,4315.32,23.234916
2,1502949600000,4330.29,4345.45,4309.37,4324.35,7.229691
3,1502953200000,4316.62,4349.99,4287.41,4349.99,4.443249
4,1502956800000,4333.32,4377.85,4333.32,4360.69,0.972807
...,...,...,...,...,...,...
51404,1688364000000,30673.61,30678.41,30606.00,30651.25,1036.522350
51405,1688367600000,30651.25,30759.80,30636.65,30667.19,1758.796390
51406,1688371200000,30667.19,30700.92,30599.07,30640.12,1390.909010
51407,1688374800000,30640.11,30670.00,30602.29,30610.61,833.620010


In [2]:
df['date'] = pd.to_datetime(df['date'], unit = 'ms')

In [3]:
df.set_index('date', inplace=True)

In [8]:
df['Moving_avg_50'] = df['close'].rolling(50).mean()
df['Moving_avg_200'] = df['close'].rolling(200).mean()


In [15]:
df['buy_signal'] = False
df['sell_signal'] = False

df.loc[df['Moving_avg_50']>df['Moving_avg_200'],'buy_signal'] = True
df.loc[df['Moving_avg_50']<df['Moving_avg_200'],'sell_signal'] = True

In [19]:
df

,open,high,low,close,volume,Moving_avg_50,Moving_avg_200,buy_signal,sell_signal
date,,,,,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,NaN,NaN,False,False
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,NaN,NaN,False,False
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,NaN,NaN,False,False
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,NaN,NaN,False,False
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...
2023-07-03 06:00:00,30673.61,30678.41,30606.00,30651.25,1036.522350,30558.3664,30477.55635,True,False
2023-07-03 07:00:00,30651.25,30759.80,30636.65,30667.19,1758.796390,30563.1230,30478.25515,True,False
2023-07-03 08:00:00,30667.19,30700.92,30599.07,30640.12,1390.909010,30566.9202,30478.88960,True,False


In [18]:
df['sell_signal'].value_counts()

sell_signal
False    26943
True     24466
Name: count, dtype: int64

In [49]:
balance = 1000
position = None

for index, row in df.iterrows():
    # BUY side ...
    if position is None and row['buy_signal']:
        buy_price = row['close']
        usd_size = balance  # go all-in

        position = {
            'buy_price': buy_price,
            'usd_size': usd_size
        }
        print(f'At: {index} Bought BTC for ${usd_size} at the price of ${buy_price}')

    # SELL side ...
    if position is not None and row['sell_signal']:
        sell_price = row['close']
        trade_result = (sell_price - position['buy_price']) / position['buy_price']  # % return

        pnl = trade_result * position['usd_size']
        balance += pnl

        print(
            f'At: {index} Sold BTC at the price of ${sell_price} '
            f'percentage change: ({round(trade_result*100, 2)}%), '
            f'P&L: ${round(pnl, 2)}, New balance: ${round(balance, 2)}'
        )

        position = None


At: 2017-08-25 11:00:00 Bought BTC for $1000 at the price of $4361.27
At: 2017-09-04 06:00:00 Sold BTC at the price of $4459.77 percentage change: (2.26%), P&L: $22.59, New balance: $1022.59
At: 2017-09-07 21:00:00 Bought BTC for $1022.5851644131182 at the price of $4641.13
At: 2017-09-09 13:00:00 Sold BTC at the price of $4300.16 percentage change: (-7.35%), P&L: $-75.13, New balance: $947.46
At: 2017-09-19 09:00:00 Bought BTC for $947.4588776015138 at the price of $3940.2
At: 2017-09-22 21:00:00 Sold BTC at the price of $3624.21 percentage change: (-8.02%), P&L: $-75.98, New balance: $871.48
At: 2017-09-26 13:00:00 Bought BTC for $871.4760516705198 at the price of $3928.33
At: 2017-10-05 07:00:00 Sold BTC at the price of $4154.1 percentage change: (5.75%), P&L: $50.09, New balance: $921.56
At: 2017-10-06 17:00:00 Bought BTC for $921.5617492024618 at the price of $4336.16
At: 2017-10-19 19:00:00 Sold BTC at the price of $5609.0 percentage change: (29.35%), P&L: $270.52, New balance: $

In [50]:
print(f'The net balance is :{round(balance,2)}')

The net balance is :7847.25


Structured Code

In [ ]:
class BacktestCrossMA:
    def __init__(self) -> None:
        self.df = pd.DataFrame()

    def load_data(self, path):
        self.df = pd.read_csv(path)
        self.df["date"] = pd.to_datetime(self.df["date"], unit="ms")
        self.df = self.df.set_index(self.df["date"])
        del self.df["date"]

    def populate_indicators(self):
        self.df['ma50'] = self.df['close'].rolling(50).mean()
        self.df['ma200'] = self.df['close'].rolling(200).mean()

    def populate_signals(self):
        self.df['buy_signal'] = False
        self.df['sell_signal'] = False
        self.df.loc[(self.df['ma50'] > self.df['ma200']), 'buy_signal'] = True
        self.df.loc[(self.df['ma50'] < self.df['ma200']), 'sell_signal'] = True

    def run_backtest(self):
        balance = 1000
        position = None
        asset = "BTC"

        for index, row in self.df.iterrows():
            if position is None and row['buy_signal']:
                open_price = row['close']
                position = {
                    'open_price': open_price,
                    'usd_size': balance,
                }
                print(f"{index} - Buy for {balance}$ of {asset} at {open_price}$")

            elif position and row['sell_signal']:
                close_price = row['close']
                trade_result = (close_price - position['open_price']) / position['open_price']
                balance = balance + trade_result * position['usd_size']
                position = None
                print(f"{index} - Sell for {balance}$ of {asset} at {close_price}$")
                
            
        print()
        print(f"Final balance: {balance}$")



In [59]:
bt = BacktestCrossMA()
bt.load_data('/Users/tusharbhatt/Desktop/Python for Finance/BTC-USDT.csv')
bt.populate_indicators()
bt.populate_signals()
bt.run_backtest()

2017-08-25 11:00:00 - Buy for 1000$ of BTC at 4361.27$
2017-09-04 06:00:00 - Sell for 1022.5851644131182$ of BTC at 4459.77$
2017-09-07 21:00:00 - Buy for 1022.5851644131182$ of BTC at 4641.13$
2017-09-09 13:00:00 - Sell for 947.4588776015138$ of BTC at 4300.16$
2017-09-19 09:00:00 - Buy for 947.4588776015138$ of BTC at 3940.2$
2017-09-22 21:00:00 - Sell for 871.4760516705198$ of BTC at 3624.21$
2017-09-26 13:00:00 - Buy for 871.4760516705198$ of BTC at 3928.33$
2017-10-05 07:00:00 - Sell for 921.5617492024618$ of BTC at 4154.1$
2017-10-06 17:00:00 - Buy for 921.5617492024618$ of BTC at 4336.16$
2017-10-19 19:00:00 - Sell for 1192.0777488092249$ of BTC at 5609.0$
2017-10-20 15:00:00 - Buy for 1192.0777488092249$ of BTC at 5889.99$
2017-10-25 02:00:00 - Sell for 1072.4670145348434$ of BTC at 5299.0$
2017-10-27 21:00:00 - Buy for 1072.4670145348434$ of BTC at 5769.6$
2017-10-28 17:00:00 - Sell for 1059.5983769488328$ of BTC at 5700.37$
2017-10-29 21:00:00 - Buy for 1059.5983769488328$ of